# Chuẩn hóa cơ sở dữ liệu - First Normal Form (1NF)

## Câu 1. Phân tích và chỉ ra điểm nào trong bảng Transactions_Raw vi phạm chuẩn 1NF? Giải thích lý do vi phạm.
 
### Answer:
- Phần cột `stock_codes` trong bảng `Transactions_Raw` vi phạm chuẩn 1NF.
- Phần này vi phạm vì không tuân thủ quy tắc mỗi ô chỉ được chứa 1 giá trị nguyên tử ở đây là 1 ô chứa nhiều stock codes 

## Câu 2. Hãy thiết kế lại bảng dữ liệu để đảm bảo tuân thủ chuẩn 1NF.
### Answer:
- Chia lại làm 2 bảng: bảng `Transactions` và bảng `Transaction_codes`
- Bảng `Transactions` gồm 3 cột: `transaction_id`, `customer_name`, `transaction_date`
- Bảng `Transaction_codes` gồm 2 cột: `transaction_id`, `stock_codes`

## Câu 3. Viết mô tả các bảng mới (tên bảng, tên các trường, khóa chính, khóa ngoại nếu có).
### Answer:
- Bảng `Transactions`: 
  - transaction_id INT PRIMARY KEY,
  - customer_name VARCHAR(100) NOT NULL,
  - transaction_date DATE NOT NULL
- Bảng `Transaction_codes`:
  - transaction_id INT NOT NULL,
  - stock_codes VARCHAR(10) NOT NULL,
  - FOREIGN KEY (transaction_id) REFERENCES Transactions(transaction_id) ON DELETE CASCADE

## Câu 4. Sau khi chuẩn hóa, bạn hãy:
- Thêm dữ liệu vào các bảng mới theo đúng chuẩn 1NF dựa vào bảng Transactions_Raw.
- Truy vấn danh sách khách hàng và các mã cổ phiếu tương ứng mà họ đã giao
dịch.
- Cập nhật tên khách hàng Nguyen Van A thành Nguyen V. A.
- Xóa mã cổ phiếu MWG khỏi giao dịch của khách hàng Nguyen V. A.


In [14]:
import pymysql
import os
from dotenv import load_dotenv

# Load environment variables from .credential file
load_dotenv("../w7/.credential")

# Get credentials from environment variables
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cursor = conn.cursor()


In [15]:
cursor.execute("CREATE DATABASE IF NOT EXISTS w9db;")
cursor.execute("USE w9db;")


# 1. Tạo bảng Transactions
cursor.execute("""
CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INT PRIMARY KEY,
    customer_name VARCHAR(100) NOT NULL,
    transaction_date DATE NOT NULL
);
""")

# 2. Tạo bảng Transaction_codes
cursor.execute("""
CREATE TABLE IF NOT EXISTS Transaction_codes (
    transaction_id INT NOT NULL,
    stock_codes VARCHAR(10) NOT NULL,
    FOREIGN KEY (transaction_id) REFERENCES Transactions(transaction_id) ON DELETE CASCADE        
);
""")

# 3. Chèn dữ liệu vào bảng Transactions
cursor.execute("""
INSERT INTO Transactions (transaction_id, customer_name, transaction_date) VALUES
(1, 'Nguyen Van A', '2024-12-01'),
(2, 'Le Thi B', '2024-12-02'),
(3, 'Tran Van C', '2024-12-03')
ON DUPLICATE KEY UPDATE customer_name=VALUES(customer_name), transaction_date=VALUES(transaction_date);
""")

# 4. Chèn dữ liệu vào bảng Transaction_codes
cursor.execute("""
INSERT INTO Transaction_codes (transaction_id, stock_codes) VALUES
(1, 'VNM'),
(1, 'FPT'),
(1, 'MWG'),
(2, 'SSI'),
(3, 'VCB'),
(3, 'TCB')
ON DUPLICATE KEY UPDATE stock_codes=VALUES(stock_codes);
""")

# Commit the changes
conn.commit()

In [17]:
# Truy vấn danh sách khách hàng và các mã cổ phiếu tương ứng mà họ đã giao dịch.
cursor.execute("""
SELECT t.customer_name, tc.stock_codes
FROM transactions t
INNER JOIN transaction_codes tc
ON t.transaction_id = tc.transaction_id;
""")

data = cursor.fetchall()
print(data)

(('Nguyen Van A', 'VNM'), ('Nguyen Van A', 'FPT'), ('Nguyen Van A', 'MWG'), ('Le Thi B', 'SSI'), ('Tran Van C', 'VCB'), ('Tran Van C', 'TCB'))


In [19]:
# Cập nhật tên khách hàng Nguyen Van A thành Nguyen V. A.
cursor.execute("""
UPDATE transactions
SET customer_name = 'Nguyen V. A.'
WHERE transaction_id = 1;
""")

# Commit the changes
conn.commit()

cursor.execute("""
SELECT customer_name
FROM transactions;
""")

data = cursor.fetchall()
print(data)

(('Nguyen V. A.',), ('Le Thi B',), ('Tran Van C',))


In [20]:
# Xóa mã cổ phiếu MWG khỏi giao dịch của khách hàng Nguyen V. A.
cursor.execute("""
DELETE FROM transaction_codes
WHERE stock_codes = 'MWG';
""")

# Commit the changes
conn.commit()

cursor.execute("""
SELECT stock_codes
FROM transaction_codes;
""")

data = cursor.fetchall()
print(data)

(('VNM',), ('FPT',), ('SSI',), ('VCB',), ('TCB',))


## Câu 5. Câu hỏi nâng cao:
- Viết truy vấn đếm số lượng mã cổ phiếu mỗi khách hàng đã giao dịch.
- Tìm khách hàng giao dịch nhiều mã cổ phiếu nhất.

In [21]:
# Viết truy vấn đếm số lượng mã cổ phiếu mỗi khách hàng đã giao dịch.
cursor.execute("""
SELECT t.customer_name, count(t.customer_name) AS stock_buy_numbers 
FROM transactions t
INNER JOIN transaction_codes tc
ON t.transaction_id = tc.transaction_id
GROUP BY t.customer_name;
""")

data = cursor.fetchall()
print(data)

(('Nguyen V. A.', 2), ('Le Thi B', 1), ('Tran Van C', 2))


In [22]:
# Tìm khách hàng giao dịch nhiều mã cổ phiếu nhất.
cursor.execute("""
SELECT t.customer_name, count(t.customer_name) AS stock_buy_numbers 
FROM transactions t
INNER JOIN transaction_codes tc
ON t.transaction_id = tc.transaction_id
GROUP BY t.customer_name
ORDER BY stock_buy_numbers DESC
LIMIT 1;
""")

data = cursor.fetchall()
print(data)

(('Nguyen V. A.', 2),)


In [23]:
# Close Connection
cursor.close()
conn.close()